### Data Augumentation - Augraphy

#### 1) 라이브러리 다운로드 및 임포트

In [ ]:
# !pip install augraphy

In [ ]:
import os

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### 2) Augraphy - Default Pipeline
> **⚠️실행 전 주의 사항⚠️**
- 증강된 이미지를 저장할 경로를 설정합니다.
- 증강된 이미지 생성을 위해 함수 `augment_images()`의 `매개변수(s_index, e_index, n_iteration)`를 지정합니다.
    - `s_index`: 이미지 증강을 실행할 원본 이미지 시작 인덱스 (0부터 시작)
    - `e_index`: 이미지 증강을 수행할 원본 이미지 마지막 인덱스
    - `n_interation`: 이미지 증강을 몇 번 수행할 것인지 (Defualt Pipeline이므로 랜덤으로 수행됨)
- 증강된 이미지 레이블을 저장할 경로를 설정합니다. 
- 증강된 이미지 레이블을 저장할 파일명(*.csv)을 설정합니다.

In [ ]:
# 원본 이미지 경로 탐색 및 파일명 추출
fpath_load = "../data/train/"
fname_list = os.listdir(fpath_load)
fname_list.sort()
print("원본 이미지 개수: ", len(fname_list))

# 증강 이미지 저장 경로 설정
fpath_save = "../data/train_augmented_tmp"
# fpath_save = "../data/train_augmented/"
if not os.path.exists(fpath_save):
    os.makedirs(fpath_save)

In [ ]:
from augraphy import *


# 이미지 증강
def augment_images(s_index=0, e_index=1569, n_iteration=10):
    # Make Pipeline for Augraphy
    pipeline = default_augraphy_pipeline()

    # Make Batches of Images    
    image_list = []
    for i in range(s_index, e_index + 1):
        # 원본 이미지 파일명 리스트 생성
        image_list.append(cv.imread(fpath_load + fname_list[i]))
        # 원본 이미지 저장
        cv.imwrite(fpath_save + fname_list[i], image_list[i - s_index])
        print(f"원본 이미지 저장: {fpath_save}{fname_list[i]}")

    # Augument Images
    for i in range(1, n_iteration + 1):
        # 이미지 증강
        image_augmented_list = pipeline(image_list)

        for j, image in enumerate(image_augmented_list):
            # 증강 이미지 저장
            fname = fname_list[s_index + j].split(".")[0]
            fname_save = f"{fpath_save}{fname}_{i}.jpg"
            cv.imwrite(fname_save, image)
            print(f"증강 이미지 저장: {fname_save}")

    return image_augmented_list

In [ ]:
# 이미지 증강 실행
# image_augmented_list = augment_images(n_iteration=10)

In [ ]:
# 증강 이미지 레이블 저장
# fname_augmented_path = "../data/train_augmented"
fname_augmented_path = "../data/train_augmented_tmp"
fname_augmented_list = os.listdir(fname_augmented_path)
fname_augmented_list.sort()

df_train_augmented = pd.DataFrame(columns=[["ID", "target"]])
df_train_augmented["ID"] = fname_augmented_list
# print(df_train_augmented)

df_train = pd.read_csv("../data/train.csv")
df_train = df_train.set_index('ID').T.to_dict()
keys_list = list(df_train.keys())

for i in range(len(df_train_augmented)):
    df_train_augmented_ID = df_train_augmented.iloc[i]["ID"]
    df_train_augmented_ID = df_train_augmented_ID.replace("_", ".").split(".")[0] + ".jpg"

    for key, value in df_train.items():
        if df_train_augmented_ID == key:
            df_train_augmented.at[i, "target"] = value["target"]


print("원본 + 증강 이미지 개수: ", df_train_augmented.shape[0])
print(df_train_augmented.isnull().sum())

df_train_augmented.to_csv("../data/train_augmented_tmp.csv", index=False)
# df_train_augmented.to_csv("../data/train_augmented.csv", index=False)